In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import holoviews as hv
import scipy.interpolate
import pickle
import pandas as pd
import hvplot.pandas
import os
import numpy as np

In [ ]:
noise_pickles = [
     # MAPPERR PHASE DITHERING/LO OFFSET REMOVAL COMPARISON -- FOR ORCA PAPER
    ("orca_paper/outputs/20240226_105437-noise-stats.pickle", "20240226_105437: no phase dithering, no LO offset"),
    ("orca_paper/outputs/20240226_105916-noise-stats.pickle", "20240226_105916: phase dithering, no LO offset"),
    ("orca_paper/outputs/20240226_110410-noise-stats.pickle", "20240226_110410: no phase dithering, software LO offset on TX"),
    ("orca_paper/outputs/20240226_110948-noise-stats.pickle", "20240226_110948: phase dithering, software LO offset on TX")
]

In [ ]:
plots = []

for filename, label in noise_pickles:

    with open(filename, 'rb') as f:
        df = pd.DataFrame(pickle.load(f)).set_index('n_stacks')
    df["stack_noise_mean_db"] = 10*np.log10(df["stack_noise_mean"])

    p = df["stack_noise_mean_db"].hvplot.line(logx=True, grid=True, label=label) #os.path.basename(df["prefix"].iloc[0]))
    plots.append(p)


plot_noise = hv.Overlay(plots).opts(width=1000, height=500, legend_position='right', ylabel='Mean Noise Power (dB)')
plot_noise

In [ ]:
#hvplot.save(plot_noise, f"/Users/abroome/Desktop/Summit2023/figs/coherent_noise_compensation_noise_stats.html")